<a href="https://colab.research.google.com/github/Rohit-Roby/Project/blob/main/04_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("hello world")

hello world


In [18]:
!pip install --upgrade objaverse


In [19]:
import objaverse as obx
obx.__version__

'0.1.7'

In [20]:
uids = obx.load_uids()
len(uids), type(uids)

(798759, list)

In [21]:
uids[:10]

['8476c4170df24cf5bbe6967222d1a42d',
 '8ff7f1f2465347cd8b80c9b206c2781e',
 'c786b97d08b94d02a1fa3b87d2e86cf1',
 '139331da744542009f146018fd0e05f4',
 'be2c02614d774f9da672dfdc44015219',
 'efd35e7d21ac482688c294e3b6c9f74e',
 '21d5f90dbc9f4f229b0faa7b56b67f3e',
 'dcd33159a0864de388de3a08f55e604a',
 'a7ad32b5d4d84ee5a40ebbd86da4dbe4',
 '7d6a14874eed48c2b720f0d1adfe6dd9']

In [ ]:
annotations = obx.load_annotations(uids)